# After Bye


In [1]:
# Dependencies and Setup
from pandasql import sqldf
import sqlite3
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
import random
from datetime import datetime

In [4]:
# Input csv files to load
input_file_to_load = "../Resources/NFL_Basic.csv"

# Read offense, oefense, and playoffs into DataFrames
df = pd.read_csv(input_file_to_load)

In [5]:
print(f"df row count: {len(df)}")

df row count: 4208


In [6]:
df.columns

Index(['Game ID', 'Date', 'Away Team', 'Away Score', 'Away Spread Open',
       'Away Spread Open Odds', 'Away Spread Close', 'Away Spread Close Odds',
       'Away MoneyLine Open', 'Away MoneyLine Close', 'Over Open',
       'Over Open Odds', 'Over Close', 'Over Close Odds', 'Home Team',
       'Home Score', 'Home Spread Open', 'Home Spread Open Odds',
       'Home Spread Close', 'Home Spread Close Odds', 'Home MoneyLine Open',
       'Home MoneyLine Close', 'Under Open', 'Under Open Odds', 'Under Close',
       'Under Close Odds'],
      dtype='object')

In [7]:
df = df.drop(columns=['Over Open', 'Over Open Odds', 'Over Close', 'Over Close Odds', 'Under Open Odds', 'Under Close Odds'])

In [8]:
df.rename(columns={'Game ID':'GameID', 'Away Team':'AwayTeam', 'Away Score':'AwayScore', 'Away Spread Open':'AwaySpreadOpen',
       'Away Spread Open Odds':'AwaySpreadOpenOdds', 'Away Spread Close':'AwaySpreadClose', 'Away Spread Close Odds':'AwaySpreadCloseOdds',
       'Away MoneyLine Open':'AwayMoneylineOpen', 'Away MoneyLine Close':'AwayMoneylineClose', 'Home Team':'HomeTeam',
       'Home Score':'HomeScore', 'Home Spread Open':'HomeSpreadOpen', 'Home Spread Open Odds':'HomeSpreadOpenOdds',
       'Home Spread Close':'HomeSpreadClose', 'Home Spread Close Odds':'HomeSpreadCloseOdds', 'Home MoneyLine Open':'HomeMoneylineOpen',
       'Home MoneyLine Close':'HomeMoneylineClose', 'Under Open':'TotalOpen', 'Under Close':'TotalClose'
        },inplace=True)

In [9]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [10]:
df['Season'] = df['Date'].dt.isocalendar().year
df['Week'] = df['Date'].dt.isocalendar().week
df['Day'] = df['Date'].dt.isocalendar().day

In [11]:
df = df.drop(df[df.Week < 36].index)

In [12]:
df["Week"] = df["Week"]-34

In [13]:
df["Week"] = df["Week"]-1

In [14]:
df["Week"] = np.where(df["Day"] == 1, df["Week"]-1, df["Week"])
df["Week"] = np.where(df["Day"] == 2, df["Week"]-1, df["Week"])
df["Day"] = np.where(df["Day"] == 1, "Mon", df["Day"])
df["Day"] = np.where(df["Day"] == 2, "Tue", df["Day"])
df["Day"] = np.where(df["Day"] == 3, "Wed", df["Day"])
df["Day"] = np.where(df["Day"] == 4, "Thu", df["Day"])
df["Day"] = np.where(df["Day"] == 5, "Fri", df["Day"])
df["Day"] = np.where(df["Day"] == 6, "Sat", df["Day"])
df["Day"] = np.where(df["Day"] == 7, "Sun", df["Day"])

In [15]:
column_to_move = df.pop("Season")
df.insert(2, "Season", column_to_move)
column_to_move = df.pop("Week")
df.insert(3, "Week", column_to_move)
column_to_move = df.pop("Day")
df.insert(4, "Day", column_to_move)

In [16]:
df.head()

,GameID,Date,Season,Week,Day,AwayTeam,AwayScore,AwaySpreadOpen,AwaySpreadOpenOdds,AwaySpreadClose,...,HomeTeam,HomeScore,HomeSpreadOpen,HomeSpreadOpenOdds,HomeSpreadClose,HomeSpreadCloseOdds,HomeMoneylineOpen,HomeMoneylineClose,TotalOpen,TotalClose
0,66,2009-09-10,2009,2,Thu,Tennessee,10,5.5,-110.0,6.5,...,Pittsburgh,13,-5.5,-110.0,-6.5,107.0,-210,-260.0,35.0,35.5
1,67,2009-09-13,2009,2,Sun,Miami,7,4,-110.0,4,...,Atlanta,19,-4,-110.0,-4,-104.0,-215,-210.0,43.0,44.0
2,68,2009-09-13,2009,2,Sun,Kansas City,24,8.5,-110.0,13,...,Baltimore,38,-8.5,-110.0,-13,104.0,-600,-700.0,37.5,36.5
3,69,2009-09-13,2009,2,Sun,Philadelphia,38,1,-110.0,-1,...,Carolina,10,-1,-110.0,1,111.0,110,115.0,44.5,43.5
4,70,2009-09-13,2009,2,Sun,Denver,12,1,-110.0,4.5,...,Cincinnati,7,-1,-110.0,-4.5,-104.0,-215,-222.0,44.5,41.0


In [18]:
df.to_excel("../Output/output.xlsx")

In [19]:
byes_df = (sqldf('''SELECT a.Season, a.Week, a.AwayTeam as Team
                      FROM df as a 
                      WHERE a.Week > 3
                      AND NOT EXISTS
                          (SELECT *
                            FROM df as b
                            WHERE a.Season = b.Season
                            and a.Week = b.Week - 1
                            and (a.AwayTeam = b.AwayTeam or a.AwayTeam = b.HomeTeam) 
                            )
                UNION
                SELECT a.Season, a.Week, a.HomeTeam as Team
                      FROM df as a 
                      WHERE a.Week > 3
                      AND NOT EXISTS
                          (SELECT *
                            FROM df as b
                            WHERE a.Season = b.Season
                            and a.Week = b.Week - 1
                            and (a.HomeTeam = b.AwayTeam or a.HomeTeam = b.HomeTeam) 
                            )
                     '''))

In [20]:
byes_df.head()

,Season,Week,Team
0,2009,4,Arizona
1,2009,4,Atlanta
2,2009,4,Carolina
3,2009,4,Philadelphia
4,2009,5,Chicago


In [22]:
byes_df.to_excel("../Output/byes.xlsx")

In [23]:
results_df = (sqldf('''SELECT a.GameID
                    , a.Season
                    , a.Week
                    , a.Day
                    , a.AwayTeam
                    , 'W' as AwayTeamResult
                    , a.AwayScore
                    , a.AwaySpreadClose as SpreadClose
                    , a.HomeScore
                    , 'L' as HomeTeamResult
                    , a.HomeTeam
                    , a.AwayScore + a.AwaySpreadClose - a.HomeScore as WinMargin
                FROM df as a
                WHERE ( a.AwayScore + a.AwaySpreadClose > a.HomeScore + 0)
                UNION
                SELECT a.GameID
                    , a.Season
                    , a.Week
                    , a.Day
                    , a.AwayTeam
                    , 'L' as AwayTeamResult
                    , a.AwayScore
                    , a.AwaySpreadClose as SpreadClose
                    , a.HomeScore
                    , 'W' as HomeTeamResult
                    , a.HomeTeam
                    , a.HomeScore - a.AwaySpreadClose - a.AwayScore
                FROM df as a
                WHERE ( a.HomeScore - a.AwaySpreadClose > a.AwayScore + 0)
                UNION
                SELECT a.GameID
                    , a.Season
                    , a.Week
                    , a.Day
                    , a.AwayTeam
                    , 'P' as AwayTeamResult
                    , a.AwayScore
                    , a.AwaySpreadClose as SpreadClose
                    , a.HomeScore
                    , 'P' as HomeTeamResult
                    , a.HomeTeam
                    , 0.0
                FROM df as a
                WHERE ( a.HomeScore - a.AwaySpreadClose = a.AwayScore + 0)
                ORDER BY a.GameID
            '''))
results_df

,GameID,Season,Week,Day,AwayTeam,AwayTeamResult,AwayScore,SpreadClose,HomeScore,HomeTeamResult,HomeTeam,WinMargin
0,66,2009,2,Thu,Tennessee,W,10,6.5,13,L,Pittsburgh,3.5
1,67,2009,2,Sun,Miami,L,7,4,19,W,Atlanta,8.0
2,68,2009,2,Sun,Kansas City,L,24,13,38,W,Baltimore,1.0
3,69,2009,2,Sun,Philadelphia,W,38,-1,10,L,Carolina,27.0
4,70,2009,2,Sun,Denver,W,12,4.5,7,L,Cincinnati,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3666,4721292,2023,7,Sun,Arizona,L,10,8.5,20,W,Seattle,1.5
3667,4721293,2023,7,Sun,L.A. Chargers,L,17,5.5,31,W,Kansas City,8.5
3668,4721294,2023,7,Sun,Green Bay,L,17,1,19,W,Denver,1.0
3669,4721295,2023,7,Sun,Miami,L,17,3,31,W,Philadelphia,11.0


In [24]:
print(f"df row count: {len(df)}")

df row count: 3673


In [25]:
results_df.to_excel('../Output/results.xlsx')

In [26]:
byes_df

,Season,Week,Team
0,2009,4,Arizona
1,2009,4,Atlanta
2,2009,4,Carolina
3,2009,4,Philadelphia
4,2009,5,Chicago
...,...,...,...
910,2023,7,Pittsburgh
911,2023,7,San Francisco
912,2023,7,Seattle
913,2023,7,Tampa Bay


In [59]:
query_title = 'Away Favs After a Bye, Home played last week'
print(query_title)

temp_df = (sqldf('''SELECT '$query_title'
                        , count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.AwayTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose < 0
                    AND r.AwayTeamResult = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    UNION
                    SELECT 'Away Favs After a Bye, Home played last week'
                        , count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.AwayTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose < 0
                    AND r.AwayTeamResult = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    ORDER BY 1, 3 DESC
                    '''))
# First instance of display dataframe
display_df = temp_df.copy()
display_df

Away Favs After a Bye, Home played last week


,'$query_title',Count,Result
0,$query_title,27,W
1,"Away Favs After a Bye, Home played last week",30,L


In [37]:
print('Away Dogs After a Bye, Home played last week')

temp_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.AwayTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose > 0
                    AND r.AwayTeamResult = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.AwayTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose > 0
                    AND r.AwayTeamResult = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Away Dogs After a Bye, Home played last week


,Count,Result
0,65,W
1,53,L


In [30]:
print('Home Favs After a Bye, Away played last week')

test_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose > 0
                    AND r.HomeTeamResult = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.AwayTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose > 0
                    AND r.HomeTeamResult = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.AwayTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

test_df

Home Favs after a bye, Away played last week


,Count,Result
0,73,W
1,68,L


In [38]:
print('Home Dogs After a Bye, Away played last week')

temp_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose < 0
                    AND r.HomeTeamResult = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.AwayTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose < 0
                    AND r.HomeTeamResult = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.AwayTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Home Dogs After a Bye, Away played last week


,Count,Result
0,36,W
1,49,L


In [39]:
print('Favs, Both After a Bye')

temp_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE ( 
                            (r.AwayTeam = b.Team AND r.SpreadClose < 0 AND r.AwayTeamResult = "W")
                            OR
                            (r.HomeTeam = b.Team AND r.SpreadClose > 0 AND r.HomeTeamResult = "W")
                            )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE ( 
                            (r.AwayTeam = b.Team AND r.SpreadClose < 0 AND r.AwayTeamResult = "L")
                            OR
                            (r.HomeTeam = b.Team AND r.SpreadClose > 0 AND r.HomeTeamResult = "L")
                            )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Favs, Both After a Bye


,Count,Result
0,123,W
1,125,L


In [40]:
print('Away, Both After a Bye')

temp_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.AwayTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.AwayTeamResult = "W"
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.AwayTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.AwayTeamResult = "L"
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Away, Both After a Bye


,Count,Result
0,123,W
1,125,L


In [41]:
print('Home Favs, After after a Bye')

temp_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose > 0
                    AND r.HomeTeamResult = "W"
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose > 0
                    AND r.HomeTeamResult = "L"
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Home Favs, After after a Bye


,Count,Result
0,80,W
1,80,L


In [49]:
print('Home Dogs, After after a Bye')

temp_df = (sqldf('''SELECT count(*) as Count, "W" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose < 0
                    AND r.HomeTeamResult = "W"
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    UNION
                    SELECT count(*) as Count, "L" as Result
                    FROM results_df as r
                    INNER JOIN byes_df as b
                    ON r.HomeTeam = b.Team
                    AND r.Season = b.Season
                    AND r.Week = b.Week - 1
                    WHERE r.SpreadClose < 0
                    AND r.HomeTeamResult = "L"
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b2
                                     WHERE r.HomeTeam = b2.Team
                                     AND r.Season = b2.Season
                                     AND r.Week = b2.Week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM byes_df as b3
                                     WHERE r.AwayTeam = b3.Team
                                     AND r.Season = b3.Season
                                     AND r.Week = b3.Week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

display_df = pd.concat([display_df, temp_df])
display_df

Home Dogs, After after a Bye


,Count,Result
0,27,W
1,30,L
0,45,W
1,43,L
